In [3]:
from vespa.application import Vespa
from dotenv import load_dotenv
import os

load_dotenv()   

app = Vespa(
    url=os.getenv("VESPA_ENDPOINT"),
    cert=os.getenv("VESPA_CERT"),
    key=os.getenv("VESPA_KEY")
)

In [4]:
query = {
    "yql": "select content_summary_chunk, url, tenant_id, file_id "
           "from sources pefund.summary "
           "where tenant_id contains \"90da21d0-8e83-4e84-961b-e6fec8b9dafe\" "
           "AND doc_type IN ('apollo')",
}
result = app.query(body=query)
result.hits[:]


[{'id': 'index:pefund/0/8bb5524d2d6e58f6158a55a0',
  'relevance': 0.0,
  'source': 'pefund',
  'fields': {'matchfeatures': {'bm25(content_summary_chunk)': 0.0,
    'cos_sim': -0.9999876894265599,
    'max_sim': 0.0},
   'content_summary_chunk': '# PhonePe\n*Comprehensive organizational profile and funding information*\n\n## Company Overview\n**Headquarters Location**: Based in None, None, this organization operates in the heart of the business ecosystem.\n\n**Industry Sector**: The company specializes in the information technology & services industry, contributing to innovation and growth in this sector.\n\n**Company Website**: [Visit their official website](http://www.phonepe.com) to learn more about their products and services.\n\n**Establishment Date**: Founded in 2015, the company has been serving the market for 9 years.',
   'url': 'https://s3-qa-8byte.s3.ap-south-1.amazonaws.com/phonepe%2Fin%2Fphonepe_apollo.pdf',
   'tenant_id': '90da21d0-8e83-4e84-961b-e6fec8b9dafe',
   'file_i

In [22]:
import os
import json
from dotenv import load_dotenv
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from vespa.application import Vespa  # Ensure you have 'pyvespa' installed
from langchain_perplexity import ChatPerplexity
load_dotenv()


# Path to your security files

app = Vespa(
    url=os.getenv("VESPA_ENDPOINT"),
    cert=os.getenv("VESPA_CERT"),
    key=os.getenv("VESPA_KEY")
)

@tool
def execute_vespa_yql(yql_query: str):
    """
    Executes a YQL query against the Vespa AI 'pefund' index.
    Input should be a valid YQL string.
    Returns the 'hits' from the search results.
    """
    try:
        query = {"yql": yql_query}
        result = app.query(body=query)
        # Return only the relevant fields to save context window space
        return [hit['fields'] for hit in result.hits]
    except Exception as e:
        return f"Error executing Vespa query: {str(e)}"

@tool
def perplexity_deep_research(query: str):
    """
    Executes an exhaustive web-search using Perplexity's 'sonar-deep-research'.
    Use this for competitor discovery and extracting complex financial metrics.
    """
    # LangChain native wrapper for Perplexity
    # It automatically looks for the PPLX_API_KEY environment variable.
    pplx_llm = ChatPerplexity(
        model="sonar-deep-research",
        temperature=0,
        pplx_api_key=os.getenv("PERPLEXITY_API_KEY") # Uses your existing key
    )
    
    response = pplx_llm.invoke(query)
    return response.content

# --- STEP 2: Setup FileSystemBackend ---
working_dir = os.path.join(os.getcwd(), "agent_workspace")
os.makedirs(working_dir, exist_ok=True)
# Ensure the skills directory exists
os.makedirs(os.path.join(working_dir, "skills", "vespa_search"), exist_ok=True)

fs_backend = FilesystemBackend(root_dir=working_dir, virtual_mode=True)

# --- STEP 3: Define Advanced Skill Protocol ---
SKILL_INSTRUCTION = """
You are a Deep Agent with access to a local filesystem and external database tools.
LOCATION: Your skills are in './skills/'.

PROTOCOL:
1. EXPLORE: 'ls' on './skills' to find capabilities.
2. PEEK: For a relevant folder, use 'read_file' on 'SKILL.md' but only focus on the YAML frontmatter.
3. LOAD: If the 'description' in the YAML matches the user's request, read the FULL 'SKILL.md' and follow its specific instructions. 
4. QUERY: Use 'execute_vespa_yql' based on the YQL patterns found in SKILL.md.
5. STORAGE: Save final summarized findings as a .md file in the root workspace.
"""

# --- STEP 4: Initialize LLM & Agent ---
llm = ChatGoogleGenerativeAI(
    model="gemini-3-pro-preview", # Using the latest stable pro model
    temperature=0.2,        # Lower temperature for better YQL generation
    google_api_key=os.getenv("GOOGLE_API_KEY"),
)

# Combine Filesystem tools with our Vespa tool
agent_tools = [execute_vespa_yql, perplexity_deep_research]

agent_executor = create_deep_agent(
    model=llm,
    backend=fs_backend, 
    tools=agent_tools,
    system_prompt=f"You are a Financial Analyst agent.\n{SKILL_INSTRUCTION}"
)

In [23]:
# --- STEP 5: Execution ---
config = {"configurable": {"thread_id": "vespa_research_001"}}

# Example query requiring the agent to use the tenant ID you provided earlier
user_task = (
    "Search for company data for tenant '90da21d0-8e83-4e84-961b-e6fec8b9dafe' "
    "Summarize the overall findings as per financial analysis report and save it to a file named 'phonepe_report.md'."
)

user_input = {"messages": [("user", user_task)]}

In [24]:
from utils import format_message
result = agent_executor.invoke(user_input)
format_message(result['messages'])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Search for company data for tenant '90da21d0-8e83-4e84-961b-e6fec8b9dafe' Summarize the overall findings as per │
│ financial analysis report and save it to a file named 'phonepe_report.md'.                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: ls                                                                                                │
│    Args: {                                                                                                      │
│   "path": "/skills"                                                                                             │
│ }                                                                                                               │
│    ID: a32fd8f4-1a5d-4d48-8a17-5aacd213cd79                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ ['/skills/perplexity/', '/skills/vespa_search/']                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: read_file                                                                                         │
│    Args: {                                                                                                      │
│   "file_path": "/skills/vespa_search/SKILL.md"                                                                  │
│ }                                                                                                               │
│    ID: 5bf5dca3-7691-46d7-8bb0-5ed22ff4a677                                                                     │
│                                                                                                                 │
│ 🔧 Tool Call: read_file                                                                                         │
│    Args: {                                                                                                      │
│   "file_path": "/skills/perplexity/SKILL.md"                                                                    │
│ }                                                                                                               │
│    ID: 8545de17-8f6f-46bb-8b18-63b48e4c8c2a                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│      1  ---                                                                                                     │
│      2  name: vespa-company-search                                                                              │
│      3  description: Search and retrieve company profiles, funding rounds, and financial data from the Vespa AI │
│ 'pefund' index. Use this when you need specific details about a company's revenue, investors, or history for a  │
│ given tenant.                                                                                                   │
│      4  ---                                                                                                     │
│      5                                                                                                          │
│      6  # Vespa Company Search Skill                                                                            │
│      7                                                                                                          │
│      8  This skill allows the agent to execute structured queries against the Vespa AI database to extract      │
│ high-fidelity company information from the `pefund.summary` source.                                             │
│      9                                                                                                          │
│     10  ## Database Schema                                                                                      │
│     11  The data is stored in the `pefund.summary` source with the following relevant fields:                   │
│     12  - `content_summary_chunk`: Markdown text containing company details or funding history.                 │
│     13  - `url`: The source document URL (S3/PDF).                                                              │
│     14  - `tenant_id`: Mandatory UUID for data isolation.                                                       │
│     15  - `file_id`: Unique identifier for the source file.                                                     │
│     16  - `doc_type`: Category of document (e.g., 'apollo').                                                    │
│     17                                                                                                          │
│     18  ## Querying Guidelines (YQL)                                                                            │
│     19                                                                                                          │
│     20  ### 1. Mandatory Filters                                                                                │
│     21  Every query **must** include a `tenant_id` filter to ensure data privacy.                               │
│     22  - **Pattern:** `where tenant_id contains "[UUID]"`                                                      │
│     23                                                                                                          │
│     24  ### 2. Document Types                                                                                   │
│     25  Use the `doc_type` filter to narrow results. Records from the Apollo dataset should be targeted using:  │
│     26  - **Pattern:** `AND doc_type IN ('apollo')`                                                             │
│     27                                                                                                          │
│     28  ### 3. Search Logic                                                                                     │
│     29  - For broad retrieval: Use `select *` or specify the fields `content_summary_chunk, url, tenant_id`.    │
│     30  - For specific lookups: Add `AND content_summary_chunk contains "keyword"` to the `where` clause.       │
│     31                                                 

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: read_file                                                                                         │
│    Args: {                                                                                                      │
│   "file_path": "/phonepe_report.md"                                                                             │
│ }                                                                                                               │
│    ID: e1d3f01a-283c-45c7-b7af-6c9238a8a629                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│      1  # Financial Analysis Report: PhonePe                                                                    │
│      2                                                                                                          │
│      3  **Tenant ID**: 90da21d0-8e83-4e84-961b-e6fec8b9dafe                                                     │
│      4  **Date**: October 26, 2025                                                                              │
│      5                                                                                                          │
│      6  ## Executive Summary                                                                                    │
│      7  PhonePe, established in 2015, has solidified its position as a major entity within the Information      │
│ Technology & Services sector. With a robust financial backing totaling over $4.7 billion and estimated annual   │
│ revenues exceeding $600 million, the company demonstrates strong market performance and significant investor    │
│ confidence, particularly from major global players like Walmart.                                                │
│      8                                                                                                          │
│      9  ## Company Profile                                                                                      │
│     10  *   **Industry Sector**: Information Technology & Services                                              │
│     11  *   **Establishment Date**: 2015                                                                        │
│     12  *   **Operational History**: 9 years                                                                    │
│     13  *   **Website**: (http://www.phonepe.com)                                                               │
│     14  *   **Professional Presence**: [LinkedIn Profile](http://www.linkedin.com/company/phonepe-internet)     │
│     15                                                                                                          │
│     16  ## Financial Highlights                                                                                 │
│     17  *   **Total Funding Raised**: $4,761,021,312                                                            │
│     18  *   **Estimated Annual Revenue**: $604,000,000                                                          │
│     19                                                                                                          │
│     20  ## Investment & Funding Analysis                                                                        │
│     21  PhonePe has successfully secured substantial capital to fuel its growth, with a total funding amount    │
│ reaching approximately **$4.76 billion**.                                                                       │
│     22                                                                                                          │
│     23  ### Recent Strategic Funding Rounds (Q1 2023)                                                           │
│     24  The data highlights significant investment activity in early 2023, indicating continued expansion and   │
│ investor trust.                                                                                                 │
│     25                                                                                                          │
│     26  *   **March 1, 2023**                                                                                   │
│     27      *   **Amount**: $200,000,000                                                                        │
│     28      *   **Round Type**: Other                                                                           │
│     29      *   **Key Investor**: Walmart              

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: write_todos                                                                                       │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "content": "Query Vespa 'pefund' index for tenant '90da21d0-8e83-4e84-961b-e6fec8b9dafe'",                │
│       "status": "completed"                                                                                     │
│     },                                                                                                          │
│     {                                                                                                           │
│       "status": "completed",                                                                                    │
│       "content": "Analyze search results to identify the company and extract financial data"                    │
│     },                                                                                                          │
│     {                                                                                                           │
│       "status": "completed",                                                                                    │
│       "content": "Generate financial analysis report and save to 'phonepe_report.md'"                           │
│     }                                                                                                           │
│   ]                                                                                                             │
│ }                                                                                                               │
│    ID: 6ee77430-82c8-4205-94bc-26d903aade0b                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated todo list to [{'content': "Query Vespa 'pefund' index for tenant                                        │
│ '90da21d0-8e83-4e84-961b-e6fec8b9dafe'", 'status': 'completed'}, {'content': 'Analyze search results to         │
│ identify the company and extract financial data', 'status': 'completed'}, {'content': "Generate financial       │
│ analysis report and save to 'phonepe_report.md'", 'status': 'completed'}]                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The company associated with tenant ID `90da21d0-8e83-4e84-961b-e6fec8b9dafe` has been identified as             │
│ **PhonePe**.                                                                                                    │
│                                                                                                                 │
│ ### Summary of Findings                                                                                         │
│ - **Company**: PhonePe (Information Technology & Services)                                                      │
│ - **Financials**:                                                                                               │
│   - **Estimated Annual Revenue**: ~$604 Million                                                                 │
│   - **Total Funding**: ~$4.76 Billion                                                                           │
│ - **Recent Activity**: Significant funding rounds in early 2023, including a **$200M** investment from          │
│ **Walmart** and **$100M** from **Ribbit Capital** and **Tiger Global**.                                         │
│                                                                                                                 │
│ I have generated a detailed financial analysis report and saved it to `phonepe_report.md`.                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [25]:
config = {"configurable": {"thread_id": "auto_comp_analysis_2025"}}

user_task = (
    "Conduct a competitor analysis for 'PhonePe' in the Indian market. "
    "Automatically find its current top 3 rivals. Benchmark their 2024/2025 revenue, "
    "market share in UPI, and funding status. Save the final report to 'phonepe_competitor_analysis.md'."
)

user_input = {"messages": [("user", user_task)]}

In [26]:
from utils import format_message
result = agent_executor.invoke(user_input)
format_message(result['messages'])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Conduct a competitor analysis for 'PhonePe' in the Indian market. Automatically find its current top 3 rivals.  │
│ Benchmark their 2024/2025 revenue, market share in UPI, and funding status. Save the final report to            │
│ 'phonepe_competitor_analysis.md'.                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: write_todos                                                                                       │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "status": "in_progress",                                                                                  │
│       "content": "Explore available skills in ./skills"                                                         │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Identify relevant skill for competitor analysis",                                             │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "status": "pending",                                                                                      │
│       "content": "Execute competitor analysis for PhonePe using the identified skill and tools"                 │
│     },                                                                                                          │
│     {                                                                                                           │
│       "status": "pending",                                                                                      │
│       "content": "Save final report to phonepe_competitor_analysis.md"                                          │
│     }                                                                                                           │
│   ]                                                                                                             │
│ }                                                                                                               │
│    ID: b6461efb-03b6-4819-90e2-3322aeb7e464                                                                     │
│                                                                                                                 │
│ 🔧 Tool Call: ls                                                                                                │
│    Args: {                                                                                                      │
│   "path": "/skills"                                                                                             │
│ }                                                                                                               │
│    ID: 59c7fa96-a253-43fc-9dc8-f27cca46cae4                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated todo list to [{'content': 'Explore available skills in ./skills', 'status': 'in_progress'}, {'content': │
│ 'Identify relevant skill for competitor analysis', 'status': 'pending'}, {'content': 'Execute competitor        │
│ analysis for PhonePe using the identified skill and tools', 'status': 'pending'}, {'content': 'Save final       │
│ report to phonepe_competitor_analysis.md', 'status': 'pending'}]                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ ['/skills/perplexity/', '/skills/vespa_search/']                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: read_file                                                                                         │
│    Args: {                                                                                                      │
│   "file_path": "/skills/perplexity/SKILL.md",                                                                   │
│   "limit": 20                                                                                                   │
│ }                                                                                                               │
│    ID: 2f1a22dc-52fa-4dd4-97f8-560cddd39e7f                                                                     │
│                                                                                                                 │
│ 🔧 Tool Call: read_file                                                                                         │
│    Args: {                                                                                                      │
│   "limit": 20,                                                                                                  │
│   "file_path": "/skills/vespa_search/SKILL.md"                                                                  │
│ }                                                                                                               │
│    ID: 498e487e-d470-429b-b7d9-03b76a382fe3                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: read_file                                                                                         │
│    Args: {                                                                                                      │
│   "file_path": "/phonepe_competitor_analysis.md"                                                                │
│ }                                                                                                               │
│    ID: 2e09668b-525a-4934-aacb-e90a6cdb5b14                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│      1  # PhonePe Competitor Analysis (India Market)                                                            │
│      2  **Date:** Late 2025                                                                                     │
│      3  **Target Company:** PhonePe                                                                             │
│      4  **Sector:** Fintech / UPI Payments                                                                      │
│      5                                                                                                          │
│      6  ## 1. Executive Summary                                                                                 │
│      7  PhonePe maintains a dominant leadership position in India's UPI ecosystem with a 45-46% market share    │
│ and ₹7,115 Cr ($850M) in FY25 revenue, driven by aggressive expansion into tier-4+ cities. However, it faces    │
│ intense scrutiny from regulators regarding the 30% market share cap and rising competition from Google Pay      │
│ (34-35% share) and an emerging, agile challenger in Navi Technologies (3% share, growing rapidly).              │
│      8                                                                                                          │
│      9  ## 2. Competitor Landscape                                                                              │
│     10  The analysis identifies the following top 3 direct competitors based on UPI transaction volume, fintech │
│ ecosystem breadth, and capital availability:                                                                    │
│     11                                                                                                          │
│     12  1.  **Google Pay (Alphabet Inc.)**: The primary challenger with a duopoly-like hold on the market       │
│ (34-35% share). Selected for its massive android ecosystem integration, virtually unlimited capital from        │
│ Alphabet ($95B cash reserves), and direct rivalry in user experience and merchant payments.                     │
│     13  2.  **Paytm (One97 Communications)**: The pioneer of digital payments in India, currently navigating a  │
│ turnaround. Selected despite recent regulatory headwinds because it remains the third-largest player (7.6%      │
│ share), has achieved near-breakeven EBITDA, and operates a diversified financial services stack.                │
│     14  3.  **Navi Technologies**: The aggressive insurgent. Selected as the fastest-growing challenger (2.8-3% │
│ share) founded by Sachin Bansal. Navi is actively disrupting the lending and insurance space and has quickly    │
│ captured significant UPI volume to cross-sell high-margin products.                                             │
│     15                                                                                                          │
│     16  ## 3. Benchmarking Table (FY2024-2025)                                                                  │
│     17                                                                                                          │
│     18  | Metric | **PhonePe** (Target) | **Google Pay** | **Paytm** | **Navi Technologies** |                  │
│     19  | :--- | :--- | :--- | :--- | :--- |                                                                    │
│     20  | **Market Share (UPI)** | **45 - 46%** | 34 - 35% | 7.0 - 7.6% | 2.8 - 3.0% |                          │
│     21  | **Revenue (FY25)** | **₹7,115 Cr** ($850M) | N/A (India Ads: ₹31k Cr)* | ₹6,900 Cr | ₹2,271 Cr        │
│ (Finserv) |                                                                                                     │
│     22  | **Revenue Growth** | +40% YoY | N/A | -35% YoY | +19% YoY |                                           │
│     23  | **Funding Status** | **$14.5B** Valuation | P

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 🔧 Tool Call: write_todos                                                                                       │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "status": "completed",                                                                                    │
│       "content": "Load instructions from /skills/perplexity/SKILL.md"                                           │
│     },                                                                                                          │
│     {                                                                                                           │
│       "status": "completed",                                                                                    │
│       "content": "Execute Perplexity research to identify competitors and extract financial data"               │
│     },                                                                                                          │
│     {                                                                                                           │
│       "status": "completed",                                                                                    │
│       "content": "Synthesize findings into final report structure"                                              │
│     },                                                                                                          │
│     {                                                                                                           │
│       "status": "completed",                                                                                    │
│       "content": "Save final report to phonepe_competitor_analysis.md"                                          │
│     }                                                                                                           │
│   ]                                                                                                             │
│ }                                                                                                               │
│    ID: 2faa06ad-60ec-4aac-9468-40a35b591edf                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated todo list to [{'content': 'Load instructions from /skills/perplexity/SKILL.md', 'status': 'completed'}, │
│ {'content': 'Execute Perplexity research to identify competitors and extract financial data', 'status':         │
│ 'completed'}, {'content': 'Synthesize findings into final report structure', 'status': 'completed'},            │
│ {'content': 'Save final report to phonepe_competitor_analysis.md', 'status': 'completed'}]                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ I have completed the competitor analysis for PhonePe.                                                           │
│                                                                                                                 │
│ The final report has been saved to **`phonepe_competitor_analysis.md`**.                                        │
│                                                                                                                 │
│ **Summary of Findings:**                                                                                        │
│ *   **Top 3 Competitors Identified**: Google Pay, Paytm, and Navi Technologies.                                 │
│ *   **PhonePe's Position**: Clear market leader with **45-46% UPI market share** and **₹7,115 Cr revenue**      │
│ (FY25), valued at $14.5B.                                                                                       │
│ *   **Key Benchmarks**:                                                                                         │
│     *   **Google Pay**: Closest rival (34-35% share), backed by Alphabet's massive capital ($95B cash).         │
│     *   **Paytm**: Third largest (7.6% share), focusing on profitability and achieving near-breakeven EBITDA.   │
│     *   **Navi**: Fastest growing challenger (3% share), using UPI to cross-sell lending products.              │
│                                                                                                                 │
│ The report includes a detailed benchmarking table, moat analysis, and financial metrics for 2024/2025 as        │
│ requested.                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Langgraph + DeepAgent + Reflection

In [ ]:
import os
from typing import List, Annotated, TypedDict
from dotenv import load_dotenv

# LangChain / LangGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_models import ChatPerplexity
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph

# DeepAgent & Vespa
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from vespa.application import Vespa

load_dotenv()

# --- 1. TOOL DEFINITIONS ---

@tool
def execute_vespa_yql(yql_query: str):
    """Executes a YQL query against the Vespa AI index."""
    try:
        app = Vespa(
            url=os.getenv("VESPA_ENDPOINT"), 
            cert=os.getenv("VESPA_CERT"), 
            key=os.getenv("VESPA_KEY")
        )
        # Log the query for debugging
        print(f"Executing YQL: {yql_query}") 
        
        result = app.query(body={"yql": yql_query})
        if not result.hits:
            return "No results found for this query. Check if the ID exists and is the correct type."
            
        return [hit['fields'] for hit in result.hits]
    except Exception as e:
        return f"Vespa Error: {str(e)}. Ensure IDs are integers if the schema requires it."

@tool
def perplexity_research(query: str):
    """Deep web research via Perplexity Sonar."""
    pplx = ChatPerplexity(model="sonar-deep-research", pplx_api_key="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")
    return pplx.invoke(query).content

SKILL_INSTRUCTION = """
You are a Deep Agent with access to a local filesystem and external database tools.
LOCATION: Your skills are in './skills/'.

PROTOCOL:
1. EXPLORE: 'ls' on './skills' to find capabilities.
2. PEEK: For a relevant folder, use 'read_file' on 'SKILL.md' but only focus on the YAML frontmatter.
3. LOAD: If the 'description' in the YAML matches the user's request, read the FULL 'SKILL.md' and follow its specific instructions. 
4. QUERY: Use 'execute_vespa_yql' based on the YQL patterns found in SKILL.md.
5. STORAGE: Save final summarized findings as a .md file in the root workspace.
"""

# --- 2. STATE DEFINITION ---

class ReflexionState(TypedDict):
    messages: Annotated[List[BaseMessage], "Conversation history"]
    internal_data: str  # Data captured from Vespa
    draft: str          # Current version of the report
    critique: str       # Feedback from the audit
    iteration: int

# --- 3. NODE: DEEP AGENT (THE GATHERER) ---

def deep_agent_gatherer(state: ReflexionState):
    """Step 1: Get data from Vespa and pass it forward."""
    working_dir = os.path.join(os.getcwd(), "agent_workspace")
    fs_backend = FilesystemBackend(root_dir=working_dir, virtual_mode=True)
    
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
    
    # Create the specialized Deep Agent
    # Inside deep_agent_gatherer
    executor = create_deep_agent(
    model=llm,
    backend=fs_backend,
    tools=[execute_vespa_yql],
    system_prompt=f"You are a Financial Analyst agent.\n{SKILL_INSTRUCTION}"
    )
    
    # We invoke the deep agent once to get the internal truth
    result = executor.invoke({"messages": state['messages']})
    final_response = result["messages"][-1].content
    
    return {
        "internal_data": final_response,
        "draft": final_response, # Initial draft is just the internal data
        "iteration": 0
    }

# --- 4. NODE: THE REFLECTOR (THE EDITOR & AUDITOR) ---

def reflexion_editor(state: ReflexionState):
    """Step 2: Audit with Perplexity and Revise the Draft."""
    iteration = state.get("iteration", 0)
    current_draft = state['draft']
    internal_facts = state['internal_data']
    
    editor_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)
    
    # 1. External Audit
    audit_query = f"Verify these financial claims and find latest market context: {current_draft[:500]}"
    external_context = perplexity_research.invoke(audit_query)
    
    # 2. Reflexion Logic: Generate Critique + New Draft
    refine_prompt = f"""
    INTERNAL DATA (VESPA): {internal_facts}
    EXTERNAL RESEARCH (PERPLEXITY): {external_context}
    CURRENT DRAFT: {current_draft}
    
    Your Task:
    1. Critique the current draft for inaccuracies or missing external context.
    2. Rewrite the report to integrate both Internal Vespa data and External Perplexity insights.
    3. If the report is now perfect and verified, include the tag [FINAL_VERSION].
    """
    
    refined_output = editor_llm.invoke(refine_prompt).content
    
    return {
        "draft": refined_output,
        "critique": "Critique generated and integrated.",
        "iteration": iteration + 1
    }

# --- 5. GRAPH CONSTRUCTION ---

def should_loop(state: ReflexionState):
    # Stop if [FINAL_VERSION] is present or we've hit 3 edits
    if "[FINAL_VERSION]" in state['draft'] or state['iteration'] >= 3:
        return END
    return "reflector"

builder = StateGraph(ReflexionState)

builder.add_node("gatherer", deep_agent_gatherer)
builder.add_node("reflector", reflexion_editor)

builder.set_entry_point("gatherer")

# Logic: Gatherer (Deep Agent) -> Reflector (Reflexion Loop) -> End/Loop
builder.add_edge("gatherer", "reflector")
builder.add_conditional_edges("reflector", should_loop)

graph = builder.compile()

# --- 6. EXECUTION ---

if __name__ == "__main__":
    task = "Prepare a financial report for tenant '90da21d0-8e83-4e84-961b-e6fec8b9dafe'."
    inputs = {"messages": [HumanMessage(content=task)]}
    
    for event in graph.stream(inputs):
        for node, values in event.items():
            print(f"\n>>> Current Node: {node}")
            if "draft" in values:
                print(f"Draft Preview: {values['draft'][:300]}...")